In [4]:
import pandas as pd
import numpy as np
import glob

A base de atendimentos dos Procons nos dará uma visão sobre o reflexo do comportamento das empresas em relação a seus clientes. É natural que haja chamados de reclamações ou dúvidas dos consumidores em relação às empresas, mas um volume alto (acima da media, por exemplo) pode indicar um comportamento desviante na postura da empresa frente à sociedade.

In [66]:
# IMPORTAÇÃO DOS ARQUIVOS CSV DOS ATENDIMENTOS DE PROCONS DE 2020

path = r'C:\DATASETS\Atendimentos Procons\1-Arquivos Baixados'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    temp_df = pd.read_csv(filename, index_col=None, header=0, sep=';', encoding='UTF8', dtype="str")
    li.append(temp_df)

# Concatena o resultado no dataframe
atendimentos_df = pd.concat(li, axis=0, ignore_index=True)

Dificuldades de encontrar o encoding correto para acentuação. Testado UTF-8, UTF-16, UTF-8-SIG, ANSI, latin-1 e outros. Só deu certo após abrir os arquivos no Notepad++ e convertê-los para UTF-8.

### Processamento/Tratamento de Dados:

##### Verificando colunas e tipos:

In [67]:
atendimentos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1976923 entries, 0 to 1976922
Data columns (total 27 columns):
AnoAtendimento              object
TrimestreAtendimento        object
MesAtendimento              object
DataAtendimento             object
CodigoRegiao                object
Regiao                      object
UF                          object
CodigoTipoAtendimento       object
DescricaoTipoAtendimento    object
CodigoAssunto               object
DescricaoAssunto            object
GrupoAssunto                object
CodigoProblema              object
DescricaoProblema           object
GrupoProblema               object
SexoConsumidor              object
FaixaEtariaConsumidor       object
CEPConsumidor               object
TipoFornecedor              object
RazaoSocialSindec           object
NomeFantasiaSindec          object
CNPJ                        object
RadicalCNPJ                 object
RazaoSocialRFB              object
NomeFantasiaRFB             object
CodigoCNAEPr

##### Verificando colunas e valores:

In [68]:
atendimentos_df.describe

<bound method NDFrame.describe of         AnoAtendimento TrimestreAtendimento MesAtendimento  \
0                 2020                    1              1   
1                 2020                    1              1   
2                 2020                    1              1   
3                 2020                    1              1   
4                 2020                    1              1   
...                ...                  ...            ...   
1976918           2020                    3              9   
1976919           2020                    3              9   
1976920           2020                    3              9   
1976921           2020                    3              9   
1976922           2020                    3              9   

                 DataAtendimento CodigoRegiao        Regiao  UF  \
0        2020-01-09 13:33:24.000           02      Nordeste  PB   
1        2020-01-09 13:44:25.000           02      Nordeste  PB   
2        2020-01-09 

##### Identificando percentual de valores ausentes por coluna:

In [69]:
(atendimentos_df.isnull().sum() / atendimentos_df.shape[0] * 100).sort_values(ascending=False)

NomeFantasiaRFB             60.754162
DescricaoCNAEPrincipal      20.041347
RazaoSocialRFB              19.731623
CodigoCNAEPrincipal         19.731623
NomeFantasiaSindec          16.006036
CEPConsumidor               11.973506
RadicalCNPJ                 10.205051
CNPJ                        10.205051
RazaoSocialSindec            0.002529
CodigoRegiao                 0.000000
UF                           0.000000
Regiao                       0.000000
TrimestreAtendimento         0.000000
DataAtendimento              0.000000
MesAtendimento               0.000000
DescricaoTipoAtendimento     0.000000
CodigoTipoAtendimento        0.000000
DescricaoProblema            0.000000
CodigoAssunto                0.000000
DescricaoAssunto             0.000000
GrupoAssunto                 0.000000
CodigoProblema               0.000000
GrupoProblema                0.000000
SexoConsumidor               0.000000
FaixaEtariaConsumidor        0.000000
TipoFornecedor               0.000000
AnoAtendimen

### Limpeza e correção dos dados:

##### Limpeza e correção da coluna CNPJ:

A primeira coisa que nos interessa aqui é a coluna CNPJ (que pode ter CPF ou CNPJ, conforme descrito no dicionário de dados). Vamos então analisar a coluna:

In [70]:
atendimentos_df.CNPJ.str.len().value_counts()

14.0    1586796
13.0     154384
12.0      26315
11.0       3522
3.0        3000
6.0         412
10.0        356
9.0         184
5.0         170
4.0          37
8.0           1
Name: CNPJ, dtype: int64

Eram esperados tamanhos de 14 (cnpj) ou 11 (cpf) dígitos. Vamos ver quem são as linhas com valores diferentes disso:

Analisando primeiro os valores com comprimento de 12 e 13 dígitos:

In [72]:
atendimentos_df[(atendimentos_df.CNPJ.str.len() == 12)|(atendimentos_df.CNPJ.str.len() == 13)]

,AnoAtendimento,TrimestreAtendimento,MesAtendimento,DataAtendimento,CodigoRegiao,Regiao,UF,CodigoTipoAtendimento,DescricaoTipoAtendimento,CodigoAssunto,...,CEPConsumidor,TipoFornecedor,RazaoSocialSindec,NomeFantasiaSindec,CNPJ,RadicalCNPJ,RazaoSocialRFB,NomeFantasiaRFB,CodigoCNAEPrincipal,DescricaoCNAEPrincipal
1402267,2020,3,8,2020-08-28 06:23:40,03,Sudeste,SP,3,CIP (Carta de Informações Preliminares),89,...,8260030,1,MERCADOLIVRE.COM ATIVIDADES DE INTERNET LTDA,MERCADOLIVRE.COM.BR / MERCADOSHOPS.COM.BR / MERCA,3361252000134,03361252,MERCADOLIVRE.COM ATIVIDADES DE INTERNET LTDA,NaN,7490104,ATIVIDADES DE INTERMEDIAÇÃO E AGENCIAMENTO DE ...
1402268,2020,3,8,2020-08-26 06:35:13,03,Sudeste,SP,3,CIP (Carta de Informações Preliminares),63,...,11680000,1,BV FINANCEIRA SA CREDITO FINANCIAMENTO E INVES...,NaN,1149953000189,01149953,BV FINANCEIRA SA CREDITO FINANCIAMENTO E INVES...,NaN,6436100,"SOCIEDADES DE CRÉDITO, FINANCIAMENTO E INVESTI..."
1402290,2020,3,8,2020-08-26 06:35:15,03,Sudeste,SP,3,CIP (Carta de Informações Preliminares),236,...,1229010,1,CENTRAL DE PRODUCOES GWUP S. A,SKOPOS EDITORA,1959772000118,01959772,CENTRAL DE PRODUCOES GWUP S/A,SKOPOS EDITORA,8211300,SERVIÇOS COMBINADOS DE ESCRITÓRIO E APOIO ADMI...
1402294,2020,3,8,2020-08-26 06:35:16,03,Sudeste,SP,3,CIP (Carta de Informações Preliminares),54,...,5797300,1,BANCO CSF S/A,CARTÃO CARREFOUR/BANCO CARREFOUR/CARTÃO ATACADÃO,8357240000150,08357240,BANCO CSF S/A,NaN,6431000,"BANCOS MÚLTIPLOS, SEM CARTEIRA COMERCIAL"
1402299,2020,3,8,2020-08-26 06:35:17,03,Sudeste,SP,3,CIP (Carta de Informações Preliminares),276,...,3905050,1,DECOLAR.COM LTDA,Decolar,3563689000231,03563689,DECOLAR. COM LTDA.,NaN,7911200,AGÊNCIAS DE VIAGENS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1976914,2020,3,9,2020-09-11 10:05:45,05,Centro-oeste,MT,3,CIP (Carta de Informações Preliminares),63,...,78055756,1,FACTA INTERMEDIACAO DE NEGOCIOS LTDA,*********,1360251000140,01360251,FACTA INTERMEDIACAO DE NEGOCIOS LTDA,NaN,8291100,ATIVIDADES DE COBRANÇA E INFORMAÇÕES CADASTRAIS
1976916,2020,3,9,2020-09-11 10:37:48,05,Centro-oeste,MT,3,CIP (Carta de Informações Preliminares),259,...,78058000,1,SKY SERVICOS DE BANDA LARGA LTDA.,SKY INTERNET BANDA LARGA @,497373000110,00497373,GALAXY BRASIL LTDA.,DIRECTV,6143400,OPERADORAS DE TELEVISÃO POR ASSINATURA POR SAT...
1976917,2020,3,9,2020-09-11 10:43:49,05,Centro-oeste,MT,3,CIP (Carta de Informações Preliminares),276,...,78055756,1,TUIUTUR VIAGENS E TURISMO LTDA - ME,CVC VIAGENS,175943000155,00175943,TUIUTUR VIAGENS E TURISMO LTDA EPP,CVC VIAGENS,7911200,AGÊNCIAS DE VIAGENS
1976918,2020,3,9,2020-09-11 10:43:49,05,Centro-oeste,MT,3,CIP (Carta de Informações Preliminares),276,...,78055756,1,GOL LINHAS AEREAS S.A.,GOL @,7575651000159,07575651,VRG LINHAS AEREAS S.A.,NaN,5111100,TRANSPORTE AÉREO DE PASSAGEIROS REGULAR


Foi possível perceber que os casos de 12d e 13d são porque está faltando um ou dois zeros à esquerda na coluna CNPJ. Vamos completar:

In [77]:
atendimentos_df['CNPJ'] = np.where(np.logical_or(atendimentos_df.CNPJ.str.len() == 12, atendimentos_df.CNPJ.str.len() == 13), atendimentos_df.CNPJ.str.zfill(14), atendimentos_df.CNPJ)

In [90]:
# Conferindo resultado:
atendimentos_df.CNPJ.str.len().value_counts()

14.0    1767495
11.0       3522
3.0        3000
6.0         412
10.0        356
9.0         184
5.0         170
4.0          37
8.0           1
Name: CNPJ, dtype: int64

In [89]:
# Conferindo se os RadicalCNPJ estão 'dentro' do CNPJ:
atendimentos_df[(atendimentos_df.CNPJ.str.len()==14)&(atendimentos_df.CNPJ.str[0:8] == atendimentos_df.RadicalCNPJ.astype(str))]

,AnoAtendimento,TrimestreAtendimento,MesAtendimento,DataAtendimento,CodigoRegiao,Regiao,UF,CodigoTipoAtendimento,DescricaoTipoAtendimento,CodigoAssunto,...,CEPConsumidor,TipoFornecedor,RazaoSocialSindec,NomeFantasiaSindec,CNPJ,RadicalCNPJ,RazaoSocialRFB,NomeFantasiaRFB,CodigoCNAEPrincipal,DescricaoCNAEPrincipal
0,2020,1,1,2020-01-09 13:33:24.000,02,Nordeste,PB,9,Abertura Direta de Reclamação,54,...,58083535,1,FORTBRASIL,FIDC FORTBRASIL,29665263000100,29665263,NaN,NaN,NaN,NaN
1,2020,1,1,2020-01-09 13:44:25.000,02,Nordeste,PB,9,Abertura Direta de Reclamação,101,...,58040270,1,LOJAS RIACHUELO SA,RIACHUELO,33200056003164,33200056,LOJAS RIACHUELO SA,RIACHUELO,4781400,COMÉRCIO VAREJISTA DE ARTIGOS DO VESTUÁRIO E A...
2,2020,1,1,2020-01-09 13:44:25.000,02,Nordeste,PB,9,Abertura Direta de Reclamação,101,...,58040270,1,ASUS DO BRASIL,ASUS DO BRASIL,09509531001070,09509531,NaN,NaN,NaN,NaN
3,2020,1,1,2020-01-09 13:26:47.000,02,Nordeste,PB,9,Abertura Direta de Reclamação,185,...,58027430,1,ENERGISA DA PARAÍBA-DISTRIBUIDORA DE ENERGIA S/A,ENERGISA,09095183000140,09095183,ENERGISA PARAIBA - DISTRIBUIDORA DE ENERGIA S.A,ENERGISA,3514000,DISTRIBUIÇÃO DE ENERGIA ELÉTRICA
4,2020,1,1,2020-01-09 13:47:01.000,02,Nordeste,PB,9,Abertura Direta de Reclamação,187,...,58079140,1,OI MÓVEL,OPERADORA OI,04164616001635,04164616,TNL PCS S/A,NaN,6120501,TELEFONIA MÓVEL CELULAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1976918,2020,3,9,2020-09-11 10:43:49,05,Centro-oeste,MT,3,CIP (Carta de Informações Preliminares),276,...,78055756,1,GOL LINHAS AEREAS S.A.,GOL @,07575651000159,07575651,VRG LINHAS AEREAS S.A.,NaN,5111100,TRANSPORTE AÉREO DE PASSAGEIROS REGULAR
1976919,2020,3,9,2020-09-11 10:43:49,05,Centro-oeste,MT,3,CIP (Carta de Informações Preliminares),276,...,78055756,1,CVC BRASIL OPERADORA E AGENCIA DE VIAGENS S.A.,NaN,10760260000119,10760260,CVC BRASIL OPERADORA E AGENCIA DE VIAGENS S.A.,NaN,7911200,AGÊNCIAS DE VIAGENS
1976920,2020,3,9,2020-09-11 10:43:49,05,Centro-oeste,MT,3,CIP (Carta de Informações Preliminares),276,...,78055756,1,JCNS VIAGENS E TURISMO LTDA-ME,BOA VIAGEM TURISMO,13577245000128,13577245,JCNS VIAGENS E TURISMO LTDA - ME,BOA VIAGEM TURISMO,7911200,AGÊNCIAS DE VIAGENS
1976921,2020,3,9,2020-09-11 13:49:38,05,Centro-oeste,MT,3,CIP (Carta de Informações Preliminares),183,...,78000000,1,AGUAS CUIABA S.A. CONCES. SERV. PUB. AGUA E ES...,AGUAS CUIABA S.A @,14995581000153,14995581,CAB CUIABA S/A - CONCESSIONARIA DE SERVICOS PU...,CAB CUIABA,3600601,"CAPTAÇÃO, TRATAMENTO E DISTRIBUIÇÃO DE ÁGUA"


Pela análise acima, percebemos que a quantidade de linhas com 'CNPJ[0:8] == RadicalCNPJ' é a mesma do value_counts para 14 dígitos após completarmos com 0 à esquerda. Ou seja, 1.767.495 linhas. Significa que o complemente de 0 foi positivo.

Vamos agora avaliar os casos da coluna CNPJ com comprimento < 11 dígitos:

In [92]:
atendimentos_df[(atendimentos_df.CNPJ.str.len() < 11)].CNPJ.value_counts()

191           2999
208000100       98
776000101       40
2135            20
395510          19
              ... 
7316391464       1
37788            1
407542           1
411221           1
7736613877       1
Name: CNPJ, Length: 600, dtype: int64

O CNPJ 191 é bem conhecido, trata-se do Banco do Brasil: 00000000000191. Vamos fazer a correção:

In [93]:
atendimentos_df.CNPJ = np.where(atendimentos_df.CNPJ == '191', '00000000000191', atendimentos_df.CNPJ)

Vamos avaliar os demais casos:

Avaliando o CNPJ '208000100':

In [94]:
atendimentos_df[atendimentos_df.CNPJ == '208000100']

,AnoAtendimento,TrimestreAtendimento,MesAtendimento,DataAtendimento,CodigoRegiao,Regiao,UF,CodigoTipoAtendimento,DescricaoTipoAtendimento,CodigoAssunto,...,CEPConsumidor,TipoFornecedor,RazaoSocialSindec,NomeFantasiaSindec,CNPJ,RadicalCNPJ,RazaoSocialRFB,NomeFantasiaRFB,CodigoCNAEPrincipal,DescricaoCNAEPrincipal
1418231,2020,3,7,2020-07-02 10:43:30,05,Centro-oeste,DF,2,Simples Consulta,63,...,NaN,1,BANCO DE BRASILIA S/A,BRB,208000100,00000208,BRB BANCO DE BRASILIA SA,BRB,6422100,"BANCOS MÚLTIPLOS, COM CARTEIRA COMERCIAL"
1418260,2020,3,7,2020-07-03 11:18:04,05,Centro-oeste,DF,2,Simples Consulta,69,...,71680368,1,BANCO DE BRASILIA S/A,BRB,208000100,00000208,BRB BANCO DE BRASILIA SA,BRB,6422100,"BANCOS MÚLTIPLOS, COM CARTEIRA COMERCIAL"
1418271,2020,3,7,2020-07-03 15:34:52,05,Centro-oeste,DF,2,Simples Consulta,53,...,NaN,1,BANCO DE BRASILIA S/A,BRB,208000100,00000208,BRB BANCO DE BRASILIA SA,BRB,6422100,"BANCOS MÚLTIPLOS, COM CARTEIRA COMERCIAL"
1418311,2020,3,7,2020-07-06 20:02:18,05,Centro-oeste,DF,3,CIP (Carta de Informações Preliminares),74,...,71615000,1,BANCO DE BRASILIA S/A,BRB,208000100,00000208,BRB BANCO DE BRASILIA SA,BRB,6422100,"BANCOS MÚLTIPLOS, COM CARTEIRA COMERCIAL"
1419255,2020,3,7,2020-07-10 13:30:56,05,Centro-oeste,DF,3,CIP (Carta de Informações Preliminares),54,...,72251012,1,BANCO DE BRASILIA S/A,BRB,208000100,00000208,BRB BANCO DE BRASILIA SA,BRB,6422100,"BANCOS MÚLTIPLOS, COM CARTEIRA COMERCIAL"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851325,2020,3,9,2020-09-29 10:49:13,05,Centro-oeste,GO,6,Atendimento Preliminar,76,...,75160000,1,BRB BANCO DE BRASILIA S/A,BRB,208000100,00000208,BRB BANCO DE BRASILIA SA,BRB,6422100,"BANCOS MÚLTIPLOS, COM CARTEIRA COMERCIAL"
1859724,2020,3,8,2020-08-11 15:46:52,05,Centro-oeste,GO,3,CIP (Carta de Informações Preliminares),53,...,NaN,1,BRB BANCO DE BRASILIA S/A,BRB,208000100,00000208,BRB BANCO DE BRASILIA SA,BRB,6422100,"BANCOS MÚLTIPLOS, COM CARTEIRA COMERCIAL"
1862598,2020,3,9,2020-09-11 14:16:44,05,Centro-oeste,GO,3,CIP (Carta de Informações Preliminares),53,...,NaN,1,BRB BANCO DE BRASILIA S/A,BRB,208000100,00000208,BRB BANCO DE BRASILIA SA,BRB,6422100,"BANCOS MÚLTIPLOS, COM CARTEIRA COMERCIAL"
1879747,2020,3,7,2020-07-23 14:43:07,05,Centro-oeste,GO,6,Atendimento Preliminar,54,...,NaN,1,BRB - BANCO DE BRASILIA SA,BRB,208000100,00000208,BRB BANCO DE BRASILIA SA,BRB,6422100,"BANCOS MÚLTIPLOS, COM CARTEIRA COMERCIAL"


O caso do CNPJ 208000100 é de se completar com zeros à esquerda. Vamos fazer:

In [95]:
atendimentos_df['CNPJ'] = np.where(atendimentos_df.CNPJ == '208000100', atendimentos_df.CNPJ.str.zfill(14), atendimentos_df.CNPJ)

Avaliando o CNPJ '776000101':

In [96]:
atendimentos_df[atendimentos_df.CNPJ == '776000101']

,AnoAtendimento,TrimestreAtendimento,MesAtendimento,DataAtendimento,CodigoRegiao,Regiao,UF,CodigoTipoAtendimento,DescricaoTipoAtendimento,CodigoAssunto,...,CEPConsumidor,TipoFornecedor,RazaoSocialSindec,NomeFantasiaSindec,CNPJ,RadicalCNPJ,RazaoSocialRFB,NomeFantasiaRFB,CodigoCNAEPrincipal,DescricaoCNAEPrincipal
1419296,2020,3,7,2020-07-13 12:43:02,05,Centro-oeste,DF,3,CIP (Carta de Informações Preliminares),57,...,NaN,1,ITAU ADMINISTRADORA DE CONSÓRCIOS LTDA.,NaN,776000101,00000776,ITAU ADMINISTRADORA DE CONSORCIOS LTDA,NaN,6619399,OUTRAS ATIVIDADES AUXILIARES DOS SERVIÇOS FINA...
1430720,2020,3,8,2020-08-04 11:49:43,02,Nordeste,PE,3,CIP (Carta de Informações Preliminares),57,...,54440225,1,ITAU ADMINISTRADORA DE CONSÓRCIOS LTDA,NaN,776000101,00000776,ITAU ADMINISTRADORA DE CONSORCIOS LTDA,NaN,6619399,OUTRAS ATIVIDADES AUXILIARES DOS SERVIÇOS FINA...
1438115,2020,3,8,2020-08-31 06:19:08,03,Sudeste,SP,3,CIP (Carta de Informações Preliminares),57,...,8572200,1,ITAU ADMINISTRADORA DE CONSORCIOS LTDA,NaN,776000101,00000776,ITAU ADMINISTRADORA DE CONSORCIOS LTDA,NaN,6619399,OUTRAS ATIVIDADES AUXILIARES DOS SERVIÇOS FINA...
1484699,2020,3,9,2020-09-04 06:15:13,03,Sudeste,SP,3,CIP (Carta de Informações Preliminares),56,...,5890500,1,ITAU ADMINISTRADORA DE CONSORCIOS LTDA,NaN,776000101,00000776,ITAU ADMINISTRADORA DE CONSORCIOS LTDA,NaN,6619399,OUTRAS ATIVIDADES AUXILIARES DOS SERVIÇOS FINA...
1517443,2020,3,9,2020-09-09 06:51:24,03,Sudeste,SP,3,CIP (Carta de Informações Preliminares),57,...,5794300,1,ITAU ADMINISTRADORA DE CONSORCIOS LTDA,NaN,776000101,00000776,ITAU ADMINISTRADORA DE CONSORCIOS LTDA,NaN,6619399,OUTRAS ATIVIDADES AUXILIARES DOS SERVIÇOS FINA...
1519733,2020,3,8,2020-08-05 19:00:13,02,Nordeste,BA,2,Simples Consulta,57,...,40253580,1,ITAU ADMINISTRADORA DE CONSORCIOS LTDA,CONSÓRCIO ITAÚ,776000101,00000776,ITAU ADMINISTRADORA DE CONSORCIOS LTDA,NaN,6619399,OUTRAS ATIVIDADES AUXILIARES DOS SERVIÇOS FINA...
1564346,2020,3,7,2020-07-09 11:44:42,01,Norte,AM,3,CIP (Carta de Informações Preliminares),57,...,69023003,1,BANCO ITAU ADMINISTRADORA DE CONSORCIOS LTDA,ITAU ADMINISTRADORA DE CONSORCIOS LTDA,776000101,00000776,ITAU ADMINISTRADORA DE CONSORCIOS LTDA,NaN,6619399,OUTRAS ATIVIDADES AUXILIARES DOS SERVIÇOS FINA...
1586002,2020,3,7,2020-07-14 08:29:10,01,Norte,PA,2,Simples Consulta,53,...,68830000,1,ITAU ADMINISTRADORA DE CONSÓRCIOS LTDA,NaN,776000101,00000776,ITAU ADMINISTRADORA DE CONSORCIOS LTDA,NaN,6619399,OUTRAS ATIVIDADES AUXILIARES DOS SERVIÇOS FINA...
1587744,2020,3,8,2020-08-10 06:42:23,03,Sudeste,SP,3,CIP (Carta de Informações Preliminares),57,...,3520030,1,ITAU ADMINISTRADORA DE CONSORCIOS LTDA,NaN,776000101,00000776,ITAU ADMINISTRADORA DE CONSORCIOS LTDA,NaN,6619399,OUTRAS ATIVIDADES AUXILIARES DOS SERVIÇOS FINA...
1613688,2020,3,9,2020-09-23 07:44:38,03,Sudeste,SP,3,CIP (Carta de Informações Preliminares),57,...,6473073,1,ITAU ADMINISTRADORA DE CONSORCIOS LTDA,NaN,776000101,00000776,ITAU ADMINISTRADORA DE CONSORCIOS LTDA,NaN,6619399,OUTRAS ATIVIDADES AUXILIARES DOS SERVIÇOS FINA...


Trata-se da mesma situação de precisar completar com zeros à esquerda. Em vez de fazermos isso para cada caso, vamos fazer de forma mais ampla:

In [136]:
# Preenchendo com zero à esquerda, tomando o cuidado de verificar antes se esse preenchimento tornará o CNPJ correspondente ao RadicalCNPJ:
atendimentos_df.CNPJ = np.where((atendimentos_df.CNPJ.str.len()<11)&(atendimentos_df.CNPJ.str.zfill(14).str[0:8] == atendimentos_df.RadicalCNPJ), atendimentos_df.CNPJ.str.zfill(14), atendimentos_df.CNPJ)

Vamos ver se resolveu para todos os casos de CNPJ < 11d:

In [137]:
atendimentos_df[(atendimentos_df.CNPJ.str.len() < 11)].CNPJ.value_counts()

Series([], Name: CNPJ, dtype: int64)

Conforme esperado. Temos então apenas CNPJ 14 digítos e CPF 11 dígitos:

In [139]:
atendimentos_df.CNPJ.str.len().value_counts()

14.0    1771655
11.0       3522
Name: CNPJ, dtype: int64

Fim da correção dos tamanhos na coluna CNPJ.
Vamos agora avaliar os nulos, que conforme vimos anteriormente são 10.205051% das linhas.

In [140]:
atendimentos_df[atendimentos_df.CNPJ.isnull()]

,AnoAtendimento,TrimestreAtendimento,MesAtendimento,DataAtendimento,CodigoRegiao,Regiao,UF,CodigoTipoAtendimento,DescricaoTipoAtendimento,CodigoAssunto,...,CEPConsumidor,TipoFornecedor,RazaoSocialSindec,NomeFantasiaSindec,CNPJ,RadicalCNPJ,RazaoSocialRFB,NomeFantasiaRFB,CodigoCNAEPrincipal,DescricaoCNAEPrincipal
8,2020,1,1,2020-01-09 14:21:24.000,02,Nordeste,PB,9,Abertura Direta de Reclamação,274,...,NaN,1,UNIVERSO ONLINE S/A,UOL,NaN,NaN,NaN,NaN,NaN,NaN
9,2020,1,1,2020-01-09 14:47:35.000,02,Nordeste,PB,9,Abertura Direta de Reclamação,101,...,58015400,1,MISTORE,MISTORE,NaN,NaN,NaN,NaN,NaN,NaN
21,2020,1,1,2020-01-09 16:28:56.000,02,Nordeste,PB,9,Abertura Direta de Reclamação,54,...,58038451,1,CREDICARD MASTERCARD,CREDICARD MASTERCARD,NaN,NaN,NaN,NaN,NaN,NaN
24,2020,1,1,2020-01-09 17:18:15.000,02,Nordeste,PB,2,Simples Consulta,188,...,NaN,1,MAX LINK,MAX LINK,NaN,NaN,NaN,NaN,NaN,NaN
27,2020,1,1,2020-01-10 09:24:38.000,02,Nordeste,PB,9,Abertura Direta de Reclamação,101,...,58068453,1,SMART MIX,SMART MIX,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1976674,2020,3,7,2020-07-09 06:43:06,03,Sudeste,SP,3,CIP (Carta de Informações Preliminares),185,...,6730000,1,ELETROPAULO METROPOLITANA ELETRICIDADE DE SAO ...,ENEL DISTRIBUIÇÃO SÃO PAULO,NaN,NaN,NaN,NaN,NaN,NaN
1976675,2020,3,7,2020-07-09 06:43:07,03,Sudeste,SP,3,CIP (Carta de Informações Preliminares),185,...,5894380,1,ELETROPAULO METROPOLITANA ELETRICIDADE DE SAO ...,ENEL DISTRIBUIÇÃO SÃO PAULO,NaN,NaN,NaN,NaN,NaN,NaN
1976871,2020,3,9,2020-09-02 15:18:32,05,Centro-oeste,MT,3,CIP (Carta de Informações Preliminares),54,...,78075800,1,ATACADÃO DOS OCULOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1976877,2020,3,9,2020-09-03 11:05:17,05,Centro-oeste,MT,3,CIP (Carta de Informações Preliminares),101,...,78005300,1,PRIME TEC,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Percebemos que outras colunas auxiliares, que poderiam nos ajudar a encontrar o CNPJ, também são nulas. Conforme análise no início do notebook, possuem índice de nulos relevante: NomeFantasiaRFB: 60.754162%, RazaoSocialRFB: 19.731623%, NomeFantasiaSindec: 16.006036%.
Por questão de tempo, vamos optar por remover do dataframe as linhas que possuem CNPJ nulo:

In [142]:
atendimentos_df = atendimentos_df.dropna(subset=['CNPJ']).reset_index(drop=True)

Verificando resultados:

In [143]:
print(atendimentos_df.shape)
print(atendimentos_df.CNPJ.str.len().value_counts())

(1775177, 27)
14    1771655
11       3522
Name: CNPJ, dtype: int64


In [144]:
# Definindo formato:
atendimentos_df.CNPJ = atendimentos_df.CNPJ.astype('str')

Fim da limpeza e correção da coluna CNPJ.

Como o que nos interessa nessa base é a quantidade de atendimentos por CNPJ / CPF, não precisamos limpar ou corrigir as demais colunas.

### Consolidação dos atendimentos por CNPJ / CPF:

In [159]:
atendimentos_consolidado_df = atendimentos_df.groupby(['CNPJ']).size().reset_index(name='Atendimentos_Procon')
atendimentos_consolidado_df.describe

<bound method NDFrame.describe of                   CNPJ  Atendimentos_Procon
0       00000000000191                10846
1       00000000000272                    2
2       00000000000515                    1
3       00000000000787                    1
4       00000000001244                    1
...                ...                  ...
130791     99893525187                    2
130792     99925001153                    1
130793     99928159653                    1
130794     99932075191                    1
130795  99999999000191                    3

[130796 rows x 2 columns]>

In [155]:
atendimentos_consolidado_df.sort_values('Atendimentos_Procon', ascending=False)

,CNPJ,Atendimentos_Procon
9515,02558157000162,78023
113908,40432544000147,75455
121832,61695227000193,48058
102270,33041260065290,41789
8970,02421421000111,37583
...,...,...
54374,14853656000161,1
54372,14853312000152,1
54367,14849345000129,1
54366,14849292000146,1


In [161]:
atendimentos_df[atendimentos_df.CNPJ=='99999999000191']

,AnoAtendimento,TrimestreAtendimento,MesAtendimento,DataAtendimento,CodigoRegiao,Regiao,UF,CodigoTipoAtendimento,DescricaoTipoAtendimento,CodigoAssunto,...,CEPConsumidor,TipoFornecedor,RazaoSocialSindec,NomeFantasiaSindec,CNPJ,RadicalCNPJ,RazaoSocialRFB,NomeFantasiaRFB,CodigoCNAEPrincipal,DescricaoCNAEPrincipal
23762,2020,1,1,2020-01-09 05:45:03.000,03,Sudeste,SP,3,CIP (Carta de Informações Preliminares),194,...,8160520,1,NÃO EMITIR CIPS OU ABRIR RECLAMAÇÕES,LISTA EVITE SITES/EMPRESA NÃO LOCALIZADA,99999999000191,99999999,NaN,NaN,NaN,NaN
51115,2020,1,3,2020-03-04 06:11:39.000,03,Sudeste,SP,3,CIP (Carta de Informações Preliminares),101,...,15044160,1,NÃO EMITIR CIPS OU ABRIR RECLAMAÇÕES,LISTA EVITE SITES/EMPRESA NÃO LOCALIZADA,99999999000191,99999999,NaN,NaN,NaN,NaN
59508,2020,1,3,2020-03-04 06:20:59.000,03,Sudeste,SP,3,CIP (Carta de Informações Preliminares),194,...,8160520,1,NÃO EMITIR CIPS OU ABRIR RECLAMAÇÕES,LISTA EVITE SITES/EMPRESA NÃO LOCALIZADA,99999999000191,99999999,NaN,NaN,NaN,NaN


Foi detectado um CNPJ inválido, vamos removê-lo:

In [162]:
atendimentos_consolidado_df.drop(index=130795, inplace=True)

In [168]:
# Como a coluna CNPJ tem tanto CPF quanto CNPJ, vamos mudar o nome dela:
atendimentos_consolidado_df = atendimentos_consolidado_df.rename(columns={"CNPJ": "CPF_CNPJ"})

Ao final, temos então 130.795 empresas/pessoas físicas que constaram em algum tipo de atendimento nos PROCONs em 2020, sendo 128.352 pessoas jurídicas e 2.443 pessoas físicas, conforme mostrado abaixo:

In [172]:
atendimentos_consolidado_df.CPF_CNPJ.str.len().value_counts()

14    128352
11      2443
Name: CPF_CNPJ, dtype: int64

Vamos salvar o resultado:

In [169]:
atendimentos_consolidado_df.to_csv(r'C:\DATASETS\Atendimentos Procons\atentimentos_procon.csv', index = False)